In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Define paths to your dataset
dataset_dir = 'Dataset\Image Data base'  # Replace this with the actual path

# ImageDataGenerator for training and validation (with real-time augmentation)
train_datagen = ImageDataGenerator(
    rescale=1./255,          # Normalize pixel values to [0, 1]
    shear_range=0.2,         # Randomly shear images
    zoom_range=0.2,          # Randomly zoom images
    horizontal_flip=True,    # Randomly flip images
    validation_split=0.2     # Split data into training (80%) and validation (20%)
)

# Load the training data
train_generator = train_datagen.flow_from_directory(
    dataset_dir,                 # This is the dataset directory
    target_size=(128, 128),      # Resize images to (128x128) for input to the model
    batch_size=32,               # Load in batches of 32
    class_mode='categorical',    # Use categorical labels (one-hot encoded)
    subset='training'            # Load training data subset (80%)
)

# Load the validation data
validation_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'          # Load validation data subset (20%)
)


<>:2: SyntaxWarning: invalid escape sequence '\I'
<>:2: SyntaxWarning: invalid escape sequence '\I'
C:\Users\vignesh\AppData\Local\Temp\ipykernel_8648\3763768549.py:2: SyntaxWarning: invalid escape sequence '\I'
  dataset_dir = 'Dataset\Image Data base'  # Replace this with the actual path


Found 159755 images belonging to 58 classes.
Found 39910 images belonging to 58 classes.


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [4]:
# Define a CNN model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),  # First convolutional layer
    MaxPooling2D(pool_size=(2, 2)),                                     # First max-pooling layer

    Conv2D(64, (3, 3), activation='relu'),                             # Second convolutional layer
    MaxPooling2D(pool_size=(2, 2)),                                     # Second max-pooling layer

    Conv2D(128, (3, 3), activation='relu'),                            # Third convolutional layer
    MaxPooling2D(pool_size=(2, 2)),                                     # Third max-pooling layer

    Flatten(),                                                         # Flatten the layers
    Dense(256, activation='relu'),                                     # Fully connected dense layer
    Dropout(0.5),                                                      # Dropout to avoid overfitting
    Dense(train_generator.num_classes, activation='softmax')           # Output layer (num_classes is the number of disease categories)
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


d:\Anaconda\anaconda\envs\ML\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [7]:
# Early stopping to avoid overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Model checkpoint to save the best model (updated extension to .keras)
model_checkpoint = ModelCheckpoint('best_cnn_model.keras', monitor='val_accuracy', save_best_only=True)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=30,  # Train for 30 epochs (or more if necessary)
    callbacks=[early_stopping, model_checkpoint]
)


Epoch 1/30


d:\Anaconda\anaconda\envs\ML\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4992/4992 ━━━━━━━━━━━━━━━━━━━━ 2687s 537ms/step - accuracy: 0.6133 - loss: 1.4416 - val_accuracy: 0.8865 - val_loss: 0.3641
Epoch 2/30
   1/4992 ━━━━━━━━━━━━━━━━━━━━ 17:24 209ms/step - accuracy: 0.9062 - loss: 0.2957

d:\Anaconda\anaconda\envs\ML\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


4992/4992 ━━━━━━━━━━━━━━━━━━━━ 1s 70us/step - accuracy: 0.9062 - loss: 0.2957 - val_accuracy: 1.0000 - val_loss: 0.4499
Epoch 3/30
4992/4992 ━━━━━━━━━━━━━━━━━━━━ 1671s 335ms/step - accuracy: 0.8424 - loss: 0.5072 - val_accuracy: 0.9169 - val_loss: 0.2698
Epoch 4/30
4992/4992 ━━━━━━━━━━━━━━━━━━━━ 0s 6us/step - accuracy: 0.9062 - loss: 0.3028 - val_accuracy: 0.8333 - val_loss: 0.8115
Epoch 5/30
4992/4992 ━━━━━━━━━━━━━━━━━━━━ 1618s 324ms/step - accuracy: 0.8880 - loss: 0.3528 - val_accuracy: 0.9327 - val_loss: 0.2191
Epoch 6/30
4992/4992 ━━━━━━━━━━━━━━━━━━━━ 0s 6us/step - accuracy: 0.9062 - loss: 0.3203 - val_accuracy: 1.0000 - val_loss: 0.1007
Epoch 7/30
4992/4992 ━━━━━━━━━━━━━━━━━━━━ 1565s 313ms/step - accuracy: 0.9090 - loss: 0.2901 - val_accuracy: 0.9458 - val_loss: 0.1857
Epoch 8/30
4992/4992 ━━━━━━━━━━━━━━━━━━━━ 0s 8us/step - accuracy: 0.9688 - loss: 0.1400 - val_accuracy: 1.0000 - val_loss: 0.0854
Epoch 9/30
4992/4992 ━━━━━━━━━━━━━━━━━━━━ 1612s 323ms/step - accuracy: 0.9215 - loss:

In [8]:
# Evaluate model performance on validation data
test_loss, test_acc = model.evaluate(validation_generator)
print(f'Validation Accuracy: {test_acc}')


1248/1248 ━━━━━━━━━━━━━━━━━━━━ 177s 142ms/step - accuracy: 0.9482 - loss: 0.1815
Validation Accuracy: 0.9472813606262207


In [9]:
model.save('plant_disease_cnn_model.h5')